In [19]:
from utils import *

In [20]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="c643d8c8-4abb-47ba-808c-bf73f60161f1")

# Configure workspace
ws = Workspace.from_config(auth=interactive_auth)

In [21]:
from azureml.core.model import InferenceConfig

env = get_env(name="sentiment-analysis", packages=["tensorflow", "transformers"])

inference_config = InferenceConfig(
    environment=env,
    source_directory="code",
    entry_script="sentiment_analysis.py",
)

In [22]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [23]:
from azureml.core import Model

service = Model.deploy(
    ws,
    "sentiment-analysis",
    [],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

entry_script /Users/ckoerner/OneDrive/B17928_Mastering Azure ML 2nd Edition/chapters_authoring/code/chapter15/code/nlp.py doesn't exist. entry_script should be path relative to current working directory



Generating Docker build context.
Package creation Succeeded
Logging into Docker registry b8a9e438956b449d976a3f4cae8455fb.azurecr.io
Logging into Docker registry b8a9e438956b449d976a3f4cae8455fb.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM b8a9e438956b449d976a3f4cae8455fb.azurecr.io/azureml/azureml_b579b90219746e493abd30931f6e0833
 ---> 3a5fe3be92fa
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> e84764c6defd
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6Ijk3MmU2M2NiLTU1MGQtNGMxZS04ZGFlLTU5MjQxYjYzMzZjYSIsInJlc291cmNlR3JvdXBOYW1lIjoibWxkZW1vIiwiYWNjb3VudE5hbWUiOiJtbGRlbW93cyIsIndvcmtzcGFjZUlkIjoiYjhhOWU0MzgtOTU2Yi00NDlkLTk3NmEtM2Y0Y2FlODQ1NWZiIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 3f19c2eec0db
 ---> ff1711871710
Step 4/5 : RUN mv '/var/azureml-app/tmpjdkwk7ex.py' /var/azureml-app/main.py
 ---> Running in d864b59feafd
 ---> 4341

In [24]:
print(service.get_logs())

2022-01-02T17:51:09,041194800+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-01-02T17:51:09,043300700+00:00 - rsyslog/run 
2022-01-02T17:51:09,043277400+00:00 - iot-server/run 
2022-01-02T17:51:09,044365600+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-01-02T17:51:09,143905700+00:00 - iot-server/finish 1 0
2022-01-02T17:51:09,145350300+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (14)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 41
2022-01-02 17:51:11.180870: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /azureml-envs/azureml_82846cfedbce6dde324bf4713fce4fe8/lib:/azureml-envs/azureml_82846cfedbce6dde324bf4713fce4

In [25]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    "query": "My ML skills are quite good."
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

{'sentiment': [{'label': 'POSITIVE', 'score': 0.9998338222503662}]}


In [26]:
service.delete()

Container has been successfully cleaned up.
